**How many COVID cases will there be?**

Factors:
* COVID case growth rates
* Vaccines timeline (getting EUA, starting distribution, achieving sufficient scale)

In [60]:
import math
import random

import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime, timedelta


# Model variables
GROWTH_DERIVATIVE_FACTOR_LOW = 5
GROWTH_DERIVATIVE_FACTOR_HIGH = 30

MONTHLY_GROWTH_MODIFIER_LOW = 0.5
MONTHLY_GROWTH_MODIFIER_HIGH = 4.0

DAYS_UNTIL_PEAK_GROWTH_LOW = -30
DAYS_UNTIL_PEAK_GROWTH_HIGH = 60

MISSING_INFECTION_RATE_LOW = 5
MISSING_INFECTION_RATE_HIGH = 30

RANDOM_CASE_FLUCTUATION_LOW = -2000
RANDOM_CASE_FLUCTUATION_HIGH = 4000

DAILY_FOURTH_WAVE_CHANCE = 0.6/365
FOURTH_WAVE_LENGTH_DAYS_LOW = 30
FOURTH_WAVE_LENGTH_DAYS_HIGH = 120

million = 1000000

# These are set to roughly curve fit the estimates from
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
# https://www.metaculus.com/questions/5727/100m-people-vaccinated-against-covid-19/
# https://www.metaculus.com/questions/5767/1bn-people-vaccinated-against-covid-19/
MONTHLY_VACCINATION_DAILY_AMOUNTS = [np.round(10 * million * (1/30) * (1/12), 0),
                                     np.round(50 * million * (1/30) * (1/12), 0),
                                     np.round(100 * million * (1/30) * (1/12), 0),
                                     np.round(150 * million * (1/30) * (1/12), 0),
                                     np.round(200 * million * (1/30) * (1/12), 0),
                                     np.round(250 * million * (1/30) * (1/12), 0),
                                     np.round(300 * million * (1/30) * (1/12), 0),
                                     np.round(400 * million * (1/30) * (1/12), 0),
                                     np.round(700 * million * (1/30) * (1/12), 0),
                                     np.round(900 * million * (1/30) * (1/12), 0),
                                     np.round(1100 * million * (1/30) * (1/12), 0),
                                     np.round(1200 * million * (1/30) * (1/12), 0)]

VACCINE_DELAY_DAYS_LOW = 1
VACCINE_DELAY_DAYS_HIGH = 45

VACCINE_MODIFIER_LOW = 0.7
VACCINE_MODIFIER_HIGH = 1.2

# Simulation parameters
N_SCENARIOS = 1000
SIM_START_DATE = '2020-11-28'
SIM_END_DATE = '2022-01-01'
CURRENT_COVID_CASES = 61.8 * million
CURRENT_COVID_GROWTH = 571428
CURRENT_COVID_DERIVATIVE = 4000
Q4_2020_START_COVID = 26.02 * million

In [61]:
def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def get_metaculus_range(rangex, minx=None, maxx=None):
    lower25, mean, upper75 = np.percentile(rangex, [25, 50, 75])
    results = {}
    results['lower25'] = lower25
    results['mean'] = mean
    results['upper75'] = upper75
    if minx:
        results['minx'] = print_pct(sum([x < minx for x in rangex]) / len(rangex))
    if maxx:
        results['maxx'] = print_pct(sum([x > maxx for x in rangex]) / len(rangex))
    return results
    

def normal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        mu = (high + low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (high - mu) / normed_sigma
        return np.random.normal(mu, sigma)

    
def lognormal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        log_low = np.log(low)
        log_high = np.log(high)
        mu = (log_high + log_low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (log_high - mu) / normed_sigma
        return np.random.lognormal(mu, sigma)

In [62]:
days_until_sim_end_date = (datetime.strptime(SIM_END_DATE, '%Y-%m-%d') - datetime.strptime(SIM_START_DATE, '%Y-%m-%d')).days + 1

dec_7_2020_covid_collector = []
q4_2020_covid_collector = []
y2020_covid_collector = []
q1_2021_covid_collector = []
jan_covid_collector = []
feb_covid_collector = []
mar_covid_collector = []
apr_covid_collector = []
may_covid_collector = []
jun_covid_collector = []
jul_covid_collector = []
aug_covid_collector = []
sep_covid_collector = []
oct_covid_collector = []
nov_covid_collector = []
dec_covid_collector = []
jan_vaccinated_collector = []
feb_vaccinated_collector = []
mar_vaccinated_collector = []
apr_vaccinated_collector = []
may_vaccinated_collector = []
jun_vaccinated_collector = []
jul_vaccinated_collector = []
aug_vaccinated_collector = []
sep_vaccinated_collector = []
oct_vaccinated_collector = []
nov_vaccinated_collector = []
dec_vaccinated_collector = []


for n in range(N_SCENARIOS):
    fourth_wave = False
    today = datetime.today()
    
    covid = CURRENT_COVID_CASES
    covid_growth = CURRENT_COVID_GROWTH
    
    growth_derivative_factor = np.round(normal_sample(GROWTH_DERIVATIVE_FACTOR_LOW, GROWTH_DERIVATIVE_FACTOR_HIGH, 0.8) + 1, 1)
    if growth_derivative_factor < 0.3:
        growth_derivative_factor = 0.3
        
    monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
    if monthly_growth_modifier < 0.5:
        monthly_growth_modifier = 0.5
     
    peak_growth_days = np.round(normal_sample(DAYS_UNTIL_PEAK_GROWTH_LOW, DAYS_UNTIL_PEAK_GROWTH_HIGH, 0.8), 0)
    if peak_growth_days < 0:
        peak_growth_days = 0
    peak_growth_date = (today + timedelta(days=peak_growth_days)).date()
    
    fourth_wave_days_length = np.round(lognormal_sample(FOURTH_WAVE_LENGTH_DAYS_LOW, FOURTH_WAVE_LENGTH_DAYS_HIGH, 0.8), 0)
    if fourth_wave_days_length < 1:
        fourth_wave_days_length = 1
        
    missing_infection_rate = np.round(lognormal_sample(MISSING_INFECTION_RATE_LOW, MISSING_INFECTION_RATE_HIGH, 0.8), 1)
    if missing_infection_rate < 1:
        missing_infection_rate = 1
        
    vaccine_delay_days = np.round(lognormal_sample(VACCINE_DELAY_DAYS_LOW, VACCINE_DELAY_DAYS_HIGH, 0.8), 1)
    if vaccine_delay_days < 0:
        vaccine_delay_days = 0
        
    vaccine_modifier = np.round(normal_sample(VACCINE_MODIFIER_LOW, VACCINE_MODIFIER_HIGH, 0.8), 1)
    if vaccine_modifier < 0.3:
        vaccine_modifier = 0.3
    
    vaccine_rate = 0
    total_vaccinated = 0
    
    verbose = (n <= 4) or (n % 10 == 0)
    really_verbose = (n <= 4)
    
    if verbose:
        print('-')
        print('-')
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))
        print('Growth derivative factor: {}'.format(growth_derivative_factor))
        print('Vaccine modifier: {}'.format(vaccine_modifier))
        print('Peak growth on: {}'.format(peak_growth_date))
        print('Missing infection rate: {}'.format(missing_infection_rate))
        print('Vaccine start date: {}'.format((datetime(2021, 1, 1) + timedelta(days = vaccine_delay_days)).date()))

    for day in range(days_until_sim_end_date):
        date = str((today + timedelta(days = day)).date())
        if really_verbose:
            print('Day {}: {}'.format(day, date))
            
        # Grow vaccines
        days_since_vaccine_start = (today + timedelta(days = day) - datetime(2021, 1, 1)).days - vaccine_delay_days
        if days_since_vaccine_start >= 0:
            i = math.floor(days_since_vaccine_start / 30)
            if i >= len(MONTHLY_VACCINATION_DAILY_AMOUNTS):
                i = len(MONTHLY_VACCINATION_DAILY_AMOUNTS) - 1
            daily_vaccines = MONTHLY_VACCINATION_DAILY_AMOUNTS[i]
            old_vaccine_rate = vaccine_rate
            vaccine_rate = np.round(np.random.normal(daily_vaccines  * vaccine_modifier,
                                                     daily_vaccines  * vaccine_modifier), 0)
            if vaccine_rate < old_vaccine_rate:
                vaccine_rate = old_vaccine_rate
            total_vaccinated = np.round(total_vaccinated + vaccine_rate, 0)
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
            
        # Grow COVID
        if day > peak_growth_days and (not fourth_wave or day > fourth_wave_end):
            covid_growth -= CURRENT_COVID_DERIVATIVE / growth_derivative_factor * (1 / monthly_growth_modifier)
        else:
            covid_growth += (CURRENT_COVID_DERIVATIVE / growth_derivative_factor) * monthly_growth_modifier
        covid_growth = np.round(covid_growth, 0)
        old_covid_growth = covid_growth
        if old_covid_growth < 0:
            old_covid_growth = 0
            
        covid_growth -= np.round(vaccine_rate * 0.02, 0)
        if covid_growth < 0:
            covid_growth = 0
            
        random_case_fluctuation = np.round(normal_sample(RANDOM_CASE_FLUCTUATION_LOW, RANDOM_CASE_FLUCTUATION_HIGH, 0.8), 0)
        covid += covid_growth + random_case_fluctuation
            
        if really_verbose:
            print('...Total COVID cases now {} (+{} {} -{} *{})'.format(covid,
                                                                        old_covid_growth,
                                                                        "%+d" % random_case_fluctuation,
                                                                        np.round(vaccine_rate * 0.02, 0),
                                                                        monthly_growth_modifier))
                
        if not fourth_wave and np.random.random() <= DAILY_FOURTH_WAVE_CHANCE:
            if really_verbose:
                print('...COVID FOURTH WAVE!!!')
            fourth_wave = True
            fourth_wave_end = day + fourth_wave_days_length
            
            
        # Record
        if date == '2020-12-07':
            dec7_covid = covid
            if verbose:
                print('...COVID by 7 Dec: {}'.format(dec7_covid))
                    
        if date == '2020-12-31':
            q4_2020_covid = covid - Q4_2020_START_COVID
            if verbose:
                print('...Q4 2020 COVID: {}'.format(q4_2020_covid))
            y2020_covid = np.round(covid * missing_infection_rate, 0)
            if verbose:
                print('...Total 2020 COVID: {}'.format(covid))
                print('...Total 2020 COVID after missing transform: {}'.format(y2020_covid))
                
        if date == '2021-01-30':
            jan_covid = covid - q4_2020_covid - Q4_2020_START_COVID
            jan_vaccinated = total_vaccinated
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-02-28':
            feb_covid = covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            feb_vaccinated = total_vaccinated
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-03-31':
            mar_covid = covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            q1_2021_covid = covid - q4_2020_covid - Q4_2020_START_COVID
            mar_vaccinated = total_vaccinated
            if verbose:
                print('...Q1 2021 COVID: {}'.format(q1_2021_covid))
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-04-30':
            apr_vaccinated = total_vaccinated
            apr_covid = covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-05-31':
            may_vaccinated = total_vaccinated
            may_covid = covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-06-30':
            jun_vaccinated = total_vaccinated
            jun_covid = covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-07-31':
            jul_vaccinated = total_vaccinated
            jul_covid = covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-08-31':
            aug_vaccinated = total_vaccinated
            aug_covid = covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-09-30':
            sep_vaccinated = total_vaccinated
            sep_covid = covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-10-31':
            oct_vaccinated = total_vaccinated
            oct_covid = covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-11-30':
            nov_vaccinated = total_vaccinated
            nov_covid = covid - oct_covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
            
        if date == '2021-12-31':
            dec_vaccinated = total_vaccinated
            dec_covid = covid - nov_covid - oct_covid - sep_covid - aug_covid - jul_covid - jun_covid - may_covid - apr_covid - mar_covid - feb_covid - jan_covid - q4_2020_covid - Q4_2020_START_COVID
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
   
    dec_7_2020_covid_collector.append(dec7_covid)
    q4_2020_covid_collector.append(q4_2020_covid)
    y2020_covid_collector.append(y2020_covid)
    q1_2021_covid_collector.append(q1_2021_covid)
    jan_covid_collector.append(jan_covid)
    feb_covid_collector.append(feb_covid)
    mar_covid_collector.append(mar_covid)
    apr_covid_collector.append(apr_covid)
    may_covid_collector.append(may_covid)
    jun_covid_collector.append(jun_covid)
    jul_covid_collector.append(jul_covid)
    aug_covid_collector.append(aug_covid)
    sep_covid_collector.append(sep_covid)
    oct_covid_collector.append(oct_covid)
    nov_covid_collector.append(nov_covid)
    dec_covid_collector.append(dec_covid)
    jan_vaccinated_collector.append(jan_vaccinated)
    feb_vaccinated_collector.append(feb_vaccinated)
    mar_vaccinated_collector.append(mar_vaccinated)
    apr_vaccinated_collector.append(apr_vaccinated)
    may_vaccinated_collector.append(may_vaccinated)
    jun_vaccinated_collector.append(jun_vaccinated)
    jul_vaccinated_collector.append(jul_vaccinated)
    aug_vaccinated_collector.append(aug_vaccinated)
    sep_vaccinated_collector.append(sep_vaccinated)
    oct_vaccinated_collector.append(oct_vaccinated)
    nov_vaccinated_collector.append(nov_vaccinated)
    dec_vaccinated_collector.append(dec_vaccinated)


-
-
## SCENARIO 1/1000 ##
Growth derivative factor: 17.4
Vaccine modifier: 1.1
Peak growth on: 2020-12-04
Missing infection rate: 20.1
Vaccine start date: 2021-01-03
Day 0: 2020-12-01
...Total vaccinated now 0 (+0)
...Total COVID cases now 62369618.0 (+571727.0 -2109 -0.0 *1.3)
Day 1: 2020-12-02
...Total vaccinated now 0 (+0)
...Total COVID cases now 62942165.0 (+572026.0 +521 -0.0 *1.3)
Day 2: 2020-12-03
...Total vaccinated now 0 (+0)
...Total COVID cases now 63517168.0 (+572325.0 +2678 -0.0 *1.3)
Day 3: 2020-12-04
...Total vaccinated now 0 (+0)
...Total COVID cases now 64093425.0 (+572624.0 +3633 -0.0 *1.3)
Day 4: 2020-12-05
...Total vaccinated now 0 (+0)
...Total COVID cases now 64663492.0 (+572447.0 -2380 -0.0 *1.3)
Day 5: 2020-12-06
...Total vaccinated now 0 (+0)
...Total COVID cases now 65233189.0 (+572270.0 -2573 -0.0 *1.3)
Day 6: 2020-12-07
...Total vaccinated now 0 (+0)
...Total COVID cases now 65813057.0 (+572093.0 +7775 -0.0 *1.3)
...COVID by 7 Dec: 65813057.0
Day 7: 2020-12

...Total COVID cases now 109569879.0 (+0 +3069 -283838.0 *4.0)
Day 394: 2021-12-30
...Total vaccinated now 1563025544.0 (+14191913.0)
...Total COVID cases now 109567299.0 (+0 -2580 -283838.0 *4.0)
Day 395: 2021-12-31
...Total vaccinated now 1577217457.0 (+14191913.0)
...Total COVID cases now 109563974.0 (+0 -3325 -283838.0 *4.0)
Day 396: 2022-01-01
...Total vaccinated now 1591409370.0 (+14191913.0)
...Total COVID cases now 109565684.0 (+0 +1710 -283838.0 *1.7)
Day 397: 2022-01-02
...Total vaccinated now 1605601283.0 (+14191913.0)
...Total COVID cases now 109569108.0 (+0 +3424 -283838.0 *1.7)
Day 398: 2022-01-03
...Total vaccinated now 1619793196.0 (+14191913.0)
...Total COVID cases now 109571353.0 (+0 +2245 -283838.0 *1.7)
Day 399: 2022-01-04
...Total vaccinated now 1633985109.0 (+14191913.0)
...Total COVID cases now 109572974.0 (+0 +1621 -283838.0 *1.7)
-
-
## SCENARIO 2/1000 ##
Growth derivative factor: 11.1
Vaccine modifier: 1.1
Peak growth on: 2021-01-14
Missing infection rate: 23.

...Total COVID cases now 145462313.0 (+0 -1765 -40088.0 *2.7)
Day 211: 2021-06-30
...Total vaccinated now 96495437.0 (+2004408.0)
...Total COVID cases now 145459661.0 (+0 -2652 -40088.0 *2.7)
Day 212: 2021-07-01
...Total vaccinated now 98499845.0 (+2004408.0)
...Total COVID cases now 145462835.0 (+0 +3174 -40088.0 *1.9)
Day 213: 2021-07-02
...Total vaccinated now 100504253.0 (+2004408.0)
...Total COVID cases now 145465667.0 (+0 +2832 -40088.0 *1.9)
Day 214: 2021-07-03
...Total vaccinated now 102508661.0 (+2004408.0)
...Total COVID cases now 145464712.0 (+0 -955 -40088.0 *1.9)
Day 215: 2021-07-04
...Total vaccinated now 104513069.0 (+2004408.0)
...Total COVID cases now 145466518.0 (+0 +1806 -40088.0 *1.9)
Day 216: 2021-07-05
...Total vaccinated now 106517477.0 (+2004408.0)
...Total COVID cases now 145466653.0 (+0 +135 -40088.0 *1.9)
Day 217: 2021-07-06
...Total vaccinated now 108521885.0 (+2004408.0)
...Total COVID cases now 145465876.0 (+0 -777 -40088.0 *1.9)
Day 218: 2021-07-07
...Tot

...Total COVID cases now 145534560.0 (+0 -2294 -78145.0 *3.1)
Day 317: 2021-10-14
...Total vaccinated now 398394923.0 (+3907247.0)
...Total COVID cases now 145537348.0 (+0 +2788 -78145.0 *3.1)
Day 318: 2021-10-15
...Total vaccinated now 402302170.0 (+3907247.0)
...Total COVID cases now 145531803.0 (+0 -5545 -78145.0 *3.1)
Day 319: 2021-10-16
...Total vaccinated now 406209417.0 (+3907247.0)
...Total COVID cases now 145535258.0 (+0 +3455 -78145.0 *3.1)
Day 320: 2021-10-17
...Total vaccinated now 410116664.0 (+3907247.0)
...Total COVID cases now 145535692.0 (+0 +434 -78145.0 *3.1)
Day 321: 2021-10-18
...Total vaccinated now 414023911.0 (+3907247.0)
...Total COVID cases now 145535732.0 (+0 +40 -78145.0 *3.1)
Day 322: 2021-10-19
...Total vaccinated now 417931158.0 (+3907247.0)
...Total COVID cases now 145540728.0 (+0 +4996 -78145.0 *3.1)
Day 323: 2021-10-20
...Total vaccinated now 421838405.0 (+3907247.0)
...Total COVID cases now 145539646.0 (+0 -1082 -78145.0 *3.1)
Day 324: 2021-10-21
...T

Day 396: 2022-01-01
...Total vaccinated now 904106096.0 (+13663264.0)
...Total COVID cases now 145588284.0 (+0 -1137 -273265.0 *3.6)
Day 397: 2022-01-02
...Total vaccinated now 917769360.0 (+13663264.0)
...Total COVID cases now 145587444.0 (+0 -840 -273265.0 *3.6)
Day 398: 2022-01-03
...Total vaccinated now 931432624.0 (+13663264.0)
...Total COVID cases now 145588428.0 (+0 +984 -273265.0 *3.6)
Day 399: 2022-01-04
...Total vaccinated now 945095888.0 (+13663264.0)
...Total COVID cases now 145593479.0 (+0 +5051 -273265.0 *3.6)
-
-
## SCENARIO 3/1000 ##
Growth derivative factor: 31.3
Vaccine modifier: 0.3
Peak growth on: 2020-12-01
Missing infection rate: 12.0
Vaccine start date: 2021-01-28
Day 0: 2020-12-01
...Total vaccinated now 0 (+0)
...Total COVID cases now 62373527.0 (+571875.0 +1652 -0.0 *3.5)
Day 1: 2020-12-02
...Total vaccinated now 0 (+0)
...Total COVID cases now 62940234.0 (+571838.0 -5131 -0.0 *3.5)
Day 2: 2020-12-03
...Total vaccinated now 0 (+0)
...Total COVID cases now 6351

...

-
-
## SCENARIO 11/1000 ##
Growth derivative factor: 30.2
Vaccine modifier: 1.3
Peak growth on: 2020-12-01
Missing infection rate: 11.3
Vaccine start date: 2021-01-02
...COVID by 7 Dec: 65804832.0
...Q4 2020 COVID: 53507131.0
...Total 2020 COVID: 79527131.0
...Total 2020 COVID after missing transform: 898656580.0
...Q1 2021 COVID: 32392339.0
-
-
## SCENARIO 21/1000 ##
Growth derivative factor: 2.0
Vaccine modifier: 0.9
Peak growth on: 2021-02-05
Missing infection rate: 5.7
Vaccine start date: 2021-01-05
...COVID by 7 Dec: 66074081.0
...Q4 2020 COVID: 58381298.0
...Total 2020 COVID: 84401298.0
...Total 2020 COVID after missing transform: 481087399.0
...Q1 2021 COVID: 88455663.0
-
-
## SCENARIO 31/1000 ##
Growth derivative factor: 25.2
Vaccine modifier: 1.0
Peak growth on: 2020-12-01
Missing infection rate: 8.2
Vaccine start date: 2021-01-10
...COVID by 7 Dec: 65812099.0
...Q4 2020 COVID: 53525123.0
...Total 2020 COVID: 79545123.0
...Total 2020 COVID after missing transform: 652270009.0


-
-
## SCENARIO 251/1000 ##
Growth derivative factor: 19.2
Vaccine modifier: 0.7
Peak growth on: 2020-12-01
Missing infection rate: 3.9
Vaccine start date: 2021-01-10
...COVID by 7 Dec: 65812662.0
...Q4 2020 COVID: 53530861.0
...Total 2020 COVID: 79550861.0
...Total 2020 COVID after missing transform: 310248358.0
...Q1 2021 COVID: 43114244.0
-
-
## SCENARIO 261/1000 ##
Growth derivative factor: 13.2
Vaccine modifier: 0.8
Peak growth on: 2020-12-05
Missing infection rate: 15.3
Vaccine start date: 2021-02-04
...COVID by 7 Dec: 65840605.0
...Q4 2020 COVID: 53707635.0
...Total 2020 COVID: 79727635.0
...Total 2020 COVID after missing transform: 1219832816.0
...Q1 2021 COVID: 48386958.0
-
-
## SCENARIO 271/1000 ##
Growth derivative factor: 11.7
Vaccine modifier: 1.1
Peak growth on: 2020-12-22
Missing infection rate: 3.6
Vaccine start date: 2021-03-04
...COVID by 7 Dec: 65868775.0
...Q4 2020 COVID: 54530191.0
...Total 2020 COVID: 80550191.0
...Total 2020 COVID after missing transform: 2899806

-
-
## SCENARIO 491/1000 ##
Growth derivative factor: 15.5
Vaccine modifier: 0.9
Peak growth on: 2020-12-01
Missing infection rate: 12.0
Vaccine start date: 2021-01-05
...COVID by 7 Dec: 65807475.0
...Q4 2020 COVID: 53514882.0
...Total 2020 COVID: 79534882.0
...Total 2020 COVID after missing transform: 954418584.0
...Q1 2021 COVID: 35945498.0
-
-
## SCENARIO 501/1000 ##
Growth derivative factor: 30.6
Vaccine modifier: 0.9
Peak growth on: 2020-12-30
Missing infection rate: 7.2
Vaccine start date: 2022-08-31
...COVID by 7 Dec: 65823793.0
...Q4 2020 COVID: 53798525.0
...Total 2020 COVID: 79818525.0
...Total 2020 COVID after missing transform: 574693380.0
...Q1 2021 COVID: 52755429.0
-
-
## SCENARIO 511/1000 ##
Growth derivative factor: 17.9
Vaccine modifier: 0.9
Peak growth on: 2020-12-14
Missing infection rate: 19.8
Vaccine start date: 2021-04-18
...COVID by 7 Dec: 65836885.0
...Q4 2020 COVID: 53884893.0
...Total 2020 COVID: 79904893.0
...Total 2020 COVID after missing transform: 1582116

-
-
## SCENARIO 731/1000 ##
Growth derivative factor: 17.4
Vaccine modifier: 0.7
Peak growth on: 2021-01-06
Missing infection rate: 20.3
Vaccine start date: 2021-01-05
...COVID by 7 Dec: 65834959.0
...Q4 2020 COVID: 54039017.0
...Total 2020 COVID: 80059017.0
...Total 2020 COVID after missing transform: 1625198045.0
...Q1 2021 COVID: 44523038.0
-
-
## SCENARIO 741/1000 ##
Growth derivative factor: 36.8
Vaccine modifier: 1.1
Peak growth on: 2021-01-28
Missing infection rate: 14.0
Vaccine start date: 2021-01-11
...COVID by 7 Dec: 65810960.0
...Q4 2020 COVID: 53616408.0
...Total 2020 COVID: 79636408.0
...Total 2020 COVID after missing transform: 1114909712.0
...Q1 2021 COVID: 39949824.0
-
-
## SCENARIO 751/1000 ##
Growth derivative factor: 9.0
Vaccine modifier: 1.1
Peak growth on: 2021-01-26
Missing infection rate: 10.1
Vaccine start date: 2021-01-03
...COVID by 7 Dec: 65872872.0
...Q4 2020 COVID: 54597450.0
...Total 2020 COVID: 80617450.0
...Total 2020 COVID after missing transform: 81423

-
-
## SCENARIO 971/1000 ##
Growth derivative factor: 26.7
Vaccine modifier: 0.8
Peak growth on: 2020-12-01
Missing infection rate: 5.1
Vaccine start date: 2021-01-04
...COVID by 7 Dec: 65806232.0
...Q4 2020 COVID: 53523126.0
...Total 2020 COVID: 79543126.0
...Total 2020 COVID after missing transform: 405669943.0
...Q1 2021 COVID: 38555820.0
-
-
## SCENARIO 981/1000 ##
Growth derivative factor: 17.5
Vaccine modifier: 1.0
Peak growth on: 2020-12-17
Missing infection rate: 14.3
Vaccine start date: 2021-02-13
...COVID by 7 Dec: 65836438.0
...Q4 2020 COVID: 53900478.0
...Total 2020 COVID: 79920478.0
...Total 2020 COVID after missing transform: 1142862835.0
...Q1 2021 COVID: 50410133.0
-
-
## SCENARIO 991/1000 ##
Growth derivative factor: 13.5
Vaccine modifier: 1.2
Peak growth on: 2021-01-11
Missing infection rate: 4.4
Vaccine start date: 2021-02-09
...COVID by 7 Dec: 65836385.0
...Q4 2020 COVID: 54211135.0
...Total 2020 COVID: 80231135.0
...Total 2020 COVID after missing transform: 3530169

## Total Worldwide COVID by 7 Dec

For calibration

In [63]:
get_metaculus_range(dec_7_2020_covid_collector)

{'lower25': 65812011.5, 'mean': 65820222.5, 'upper75': 65833091.75}

## 2020 Total COVID Cases with Missing Case Estimation

For https://www.metaculus.com/questions/3892/how-many-infections-of-sars-cov-2-novel-coronavirus-will-be-estimated-to-have-occurred-worldwide-before-2021/

In [64]:
get_metaculus_range(y2020_covid_collector, minx=1000000)

{'lower25': 613764951.75,
 'mean': 1015416254.5,
 'upper75': 1576003866.25,
 'minx': '<1.0%'}

## 2020 Q4 Recorded COVID Cases

For https://www.metaculus.com/questions/3767/how-many-new-cases-of-covid-19-in-the-4th-quarter-of-2020/

In [65]:
get_metaculus_range(q4_2020_covid_collector, minx=1000, maxx=1000000000)

{'lower25': 53530357.75,
 'mean': 53684341.5,
 'upper75': 53891586.75,
 'minx': '<1.0%',
 'maxx': '<1.0%'}

## 2021 Q1 Recorded COVID Cases

For https://www.metaculus.com/questions/3768/how-many-new-cases-of-covid-19-in-the-1st-quarter-of-2021/

In [66]:
get_metaculus_range(q1_2021_covid_collector, minx=1000, maxx=1000000000)

{'lower25': 36152685.5,
 'mean': 40017021.5,
 'upper75': 46412770.25,
 'minx': '<1.0%',
 'maxx': '<1.0%'}

## Vaccinations

In [67]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

collectors = [jan_vaccinated_collector, feb_vaccinated_collector, mar_vaccinated_collector, apr_vaccinated_collector,
              may_vaccinated_collector, jun_vaccinated_collector, jul_vaccinated_collector, aug_vaccinated_collector,
              sep_vaccinated_collector, oct_vaccinated_collector, nov_vaccinated_collector, dec_vaccinated_collector]

for i, month in enumerate(months):
    print('total vaccinated by EOM {} 2021: {}'.format(month, np.round(np.array(collectors[i]).mean()), 0))

total vaccinated by EOM Jan 2021: 1254122.0
total vaccinated by EOM Feb 2021: 7851812.0
total vaccinated by EOM Mar 2021: 24051754.0
total vaccinated by EOM Apr 2021: 50653007.0
total vaccinated by EOM May 2021: 90152721.0
total vaccinated by EOM Jun 2021: 140486909.0
total vaccinated by EOM Jul 2021: 205488765.0
total vaccinated by EOM Aug 2021: 288657766.0
total vaccinated by EOM Sep 2021: 416726639.0
total vaccinated by EOM Oct 2021: 603925758.0
total vaccinated by EOM Nov 2021: 835370498.0
total vaccinated by EOM Dec 2021: 1118142420.0


In [68]:
# https://www.metaculus.com/questions/4066/10m-are-administered-an-efficacious-vaccine/
for i, month in enumerate(months):
    print('10M vaccinated by EOM {} 2021: {}'.format(month, print_pct(np.sum([x > 10 * million for x in collectors[i]]) / len(collectors[i]))))
    
print('-')
for i, month in enumerate(months):
    print('100M vaccinated by EOM {} 2021: {}'.format(month, print_pct(np.sum([x > 100 * million for x in collectors[i]]) / len(collectors[i]))))

print('-')
for i, month in enumerate(months):
    print('1B vaccinated by EOM {} 2021: {}'.format(month, print_pct(np.sum([x > 1000 * million for x in collectors[i]]) / len(collectors[i]))))

10M vaccinated by EOM Jan 2021: <1.0%
10M vaccinated by EOM Feb 2021: 34.0%
10M vaccinated by EOM Mar 2021: 86.0%
10M vaccinated by EOM Apr 2021: 94.0%
10M vaccinated by EOM May 2021: 96.0%
10M vaccinated by EOM Jun 2021: 98.0%
10M vaccinated by EOM Jul 2021: 98.0%
10M vaccinated by EOM Aug 2021: 99.0%
10M vaccinated by EOM Sep 2021: 99.0%
10M vaccinated by EOM Oct 2021: 99.0%
10M vaccinated by EOM Nov 2021: >99.0%
10M vaccinated by EOM Dec 2021: >99.0%
-
100M vaccinated by EOM Jan 2021: <1.0%
100M vaccinated by EOM Feb 2021: <1.0%
100M vaccinated by EOM Mar 2021: <1.0%
100M vaccinated by EOM Apr 2021: <1.0%
100M vaccinated by EOM May 2021: 40.0%
100M vaccinated by EOM Jun 2021: 84.0%
100M vaccinated by EOM Jul 2021: 93.0%
100M vaccinated by EOM Aug 2021: 96.0%
100M vaccinated by EOM Sep 2021: 98.0%
100M vaccinated by EOM Oct 2021: 98.0%
100M vaccinated by EOM Nov 2021: 99.0%
100M vaccinated by EOM Dec 2021: 99.0%
-
1B vaccinated by EOM Jan 2021: <1.0%
1B vaccinated by EOM Feb 2021: <1

## 2021 Peak Month

For https://www.metaculus.com/questions/5432/month-with-peak-of-covid-19-cases-in-2021/

In [69]:
max_month_collector = []

for i in range(N_SCENARIOS):
    covid_months = [jan_covid_collector[i], feb_covid_collector[i], mar_covid_collector[i], apr_covid_collector[i],
                    may_covid_collector[i], jun_covid_collector[i], jul_covid_collector[i], aug_covid_collector[i],
                    sep_covid_collector[i], oct_covid_collector[i], nov_covid_collector[i], dec_covid_collector[i]]
    max_month = months[covid_months.index(max(covid_months))]
    max_month_collector.append(max_month)

pd.Series(max_month_collector).value_counts(normalize=True) * 100

Jan    91.9
Mar     5.9
Feb     1.0
May     0.4
Jul     0.2
Jun     0.2
Apr     0.2
Aug     0.1
Oct     0.1
dtype: float64